In [1]:
import heapq
from typing import Optional

from lib.altscoreapi_helper import AltScoreApiHelper
from lib.constants import Constants

In [2]:
token = Constants.API_KEY
api = AltScoreApiHelper(token)

### Some necessary methods and definitions

In [3]:
# Conversions
row_conversion = {
    "1":7, "2":6, "3":5, "4":4, "5":3, "6":2, "7":1, "8":0
}
col_conversion = {
    "a":0, "b":1, "c":2, "d":3, "e":4, "f":5, "g":6, "h":7
}
inverse_row_conversion = {
    v: k for k, v in row_conversion.items()
}
inverse_col_conversion = {
    v: k for k, v in col_conversion.items()
}


In [4]:
def convert_message_to_map(message: str) :
    rows = message.split('|')
    map = [["." for _ in range(8)] for _ in range(8)]
    enemy = (0,0)
    ally = (0,0)
    for row in rows :
        cols = [row[i:i+3] for i in range(0, len(row), 3)]
        for col in cols :
            if col[1] == "0":
                continue
            x,y = col_conversion[col[0]], row_conversion[col[2]]
            if col[1] == "^" :
                enemy = (x, y)
                map[y][x] = "E"
            elif col[1] == "#" :
                ally = (x, y)
                map[y][x] = "A"
            else:
                map[y][x] = col[1]
    return map, ally, enemy

## Dijkstra Algorithm

In [5]:
def get_neighbors(pos, map):
    x, y = pos
    neighbors = []
    directions = [(0, 1), (0, -1), (1, 0), (-1, 0)]
    for dx, dy in directions:
        nx, ny = x + dx, y + dy
        if 0 <= nx < len(map) and 0 <= ny < len(map[0]):
            if map[ny][nx] != "$":
                neighbors.append((nx, ny))
    return neighbors

def dijkstra_path(map, ally, enemy, neighbors_function=get_neighbors):
    came_from = {}
    frontier = []
    heapq.heappush(frontier, (0, enemy))
    g_score = {enemy: 0}

    while frontier:
        current_cost, current = heapq.heappop(frontier)

        if current == ally:
            path = [current]
            while current in came_from:
                current = came_from[current]
                path.append(current)
            path.reverse()
            return path

        for neighbor in neighbors_function(current, map):
            tent_g = g_score[current] + 1
            if neighbor not in g_score or tent_g < g_score[neighbor]:
                g_score[neighbor] = tent_g
                came_from[neighbor] = current
                heapq.heappush(frontier, (tent_g, neighbor))

    return None

### Detection of new neighbor search function

In [6]:
def detect_movement_pattern(prev_position, current_position):
    delta = [prev_position[0] - current_position[0], prev_position[1] - current_position[1]]
    dx, dy = delta
    return {
        (dx, dy), (dx, -dy),
        (-dx, dy), (-dx, -dy),
        (dy, dx), (dy, -dx),
        (-dy, dx), (-dy, -dx)
    }

def custom_neighbors_function(movement_set):
    return lambda pos, map: [
        (pos[0] + dx, pos[1] + dy)
        for dx, dy in movement_set
        if 0 <= pos[0] + dx < len(map[0])
        and 0 <= pos[1] + dy < len(map)
        and map[pos[1] + dy][pos[0] + dx] != "$"
    ]

## Prediction Algorithm

In [7]:
def predict_next_movement(
        message: str,
        prev_message: Optional[str] = None,
        verbose: bool = False
):
    map, ally, enemy = convert_message_to_map(message)
    next_movement_prediction = None

    if prev_message is not None:
        prev_map, prev_ally, prev_enemy = convert_message_to_map(prev_message)
        pos = detect_movement_pattern(prev_enemy, enemy)
        possible_path = dijkstra_path(
            map=map, ally=ally, enemy=enemy,
            neighbors_function=custom_neighbors_function(pos)
        )
        if enemy in dijkstra_path(
            map=prev_map, ally=prev_ally, enemy=prev_enemy,
            neighbors_function=custom_neighbors_function(pos)
        ):
            next_movement_prediction = possible_path[1]
    else:
        possible_path = dijkstra_path(map=map, ally=ally, enemy=enemy)

    if verbose:
        for i, r in enumerate(map):
            _r = []
            for j, c in enumerate(r):
                if (j,i) != ally and (j,i) != enemy and (j,i) in possible_path:
                    _r.append("*")
                else:
                    _r.append(c)
            print(_r)
        print(f"Possible Path: {possible_path}")

    return next_movement_prediction

## Example map display

In [8]:
test_position_message = "a01b01c01d01e01f01g01h01|a02b02c02d02e$2f02g02h02|a03b03c03d03e03f03g03h$3|a04b04c04d04e04f04g04h04|a05b05c05d05e$5f05g^5h05|a06b06c06d06e$6f06g06h06|a07b07c07d07e07f07g07h07|a08b08c08d08e08f#8g08h08|"

In [9]:
predict_next_movement(test_position_message, verbose=True)

['.', '.', '.', '.', '.', 'A', '.', '.']
['.', '.', '.', '.', '.', '*', '.', '.']
['.', '.', '.', '.', '$', '*', '.', '.']
['.', '.', '.', '.', '$', '*', 'E', '.']
['.', '.', '.', '.', '.', '.', '.', '.']
['.', '.', '.', '.', '.', '.', '.', '$']
['.', '.', '.', '.', '$', '.', '.', '.']
['.', '.', '.', '.', '.', '.', '.', '.']
Possible Path: [(6, 3), (5, 3), (5, 2), (5, 1), (5, 0)]


## We started the Challenge

In [10]:
api.s1_e5_start_challenge()

### Perform max 3 turns

In [11]:
prev_message = None

messages = [
    "a01b^1c01d01e01f01g01h01|a02b02c02d$2e02f02g02h02|a03b03c$3d03e03f03g03h03|a04b04c$4d04e04f04g04h04|a05b05c05d05e05f05g05h05|a06b06c06d$6e06f06g06h06|a07b07c07d07e07f07g07h07|a08b08c08d08e#8f08g08h08|",
    "a01b01c01d01e01f01g01h01|a02b02c02d$2e02f02g02h02|a^3b03c$3d03e03f03g03h03|a04b04c$4d04e04f04g04h04|a05b05c05d05e05f05g05h05|a06b06c06d$6e06f06g06h06|a07b07c07d07e07f07g07h07|a08b08c08d08e#8f08g08h08|",
    "a01b01c01d01e01f01g01h01|a02b02c02d$2e02f02g02h02|a03b03c$3d03e03f03g03h03|a04b04c$4d04e04f04g04h04|a05b^5c05d05e05f05g05h05|a06b06c06d$6e06f06g06h06|a07b07c07d07e07f07g07h07|a08b08c08d08e#8f08g08h08|",
    "a01b01c01d01e01f01g01h01|a02b02c02d$2e02f02g02h02|a03b03c$3d03e03f03g03h03|a04b04c$4d04e04f04g04h04|a05b05c05d05e05f05g05h05|a06b06c06d$6e06f06g06h06|a07b07c^7d07e07f07g07h07|a08b08c08d08e#8f08g08h08|"
]

next_movement = 0

# It is only attempted 3 times at most
for i in range(3):
    turns_remaining, timing_remaining, message = api.s1_e5_get_radar_read()
    # message = messages[i]
    next_movement = predict_next_movement(message, prev_message, verbose=True)
    if next_movement is not None:
        next_movement = {
            "x": inverse_col_conversion[next_movement[0]],
            "y": inverse_row_conversion[next_movement[1]]
        }
    prev_message = message
    print(f"Next movement prediction: {next_movement}")
    print()

['.', '*', '*', '*', 'A', '.', '.', '.']
['.', '*', '.', '.', '.', '.', '.', '.']
['.', '*', '.', '$', '.', '.', '.', '.']
['.', '*', '.', '.', '.', '.', '.', '.']
['.', '*', '$', '.', '.', '.', '.', '.']
['.', '*', '$', '.', '.', '.', '.', '.']
['.', '*', '.', '$', '.', '.', '.', '.']
['.', 'E', '.', '.', '.', '.', '.', '.']
Possible Path: [(1, 7), (1, 6), (1, 5), (1, 4), (1, 3), (1, 2), (1, 1), (1, 0), (2, 0), (3, 0), (4, 0)]
Next movement prediction: None

['.', '.', '.', '.', 'A', '.', '.', '.']
['.', '.', '*', '.', '.', '.', '.', '.']
['.', '.', '.', '$', '.', '.', '.', '.']
['.', '*', '.', '.', '.', '.', '.', '.']
['.', '.', '$', '.', '.', '.', '.', '.']
['E', '.', '$', '.', '.', '.', '.', '.']
['.', '.', '.', '$', '.', '.', '.', '.']
['.', '.', '.', '.', '.', '.', '.', '.']
Possible Path: [(0, 5), (1, 3), (2, 1), (4, 0)]
Next movement prediction: {'x': 'b', 'y': '5'}

['.', '.', '.', '.', 'A', '.', '.', '.']
['.', '.', '*', '.', '.', '.', '.', '.']
['.', '.', '.', '$', '.', '.',

In [12]:
if api.s1_e5_send_enemy_prediction(next_movement):
    print("Success")
else:
    print("Failed")

In [13]:
#Solution
sol = {
    "action": "attack",
    "attack_position": {
        "x":"c", "y":7
    }
}